# 06 – Negative Binomial GLM (Cyclists)

Goal:
Estimate the effect of darkness on cyclist counts,
controlling for hour-of-day, weekday, and seasonality.

We compare:
- Poisson GLM
- Negative Binomial GLM

We check:
- Overdispersion
- Model fit
- Effect size (% change)

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

/Users/shavarshmelikyan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/shavarshmelikyan/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
BIG_PATH = "../data/mdm2_data_files/big_table.csv"

big = pd.read_csv(BIG_PATH)
big["datetime"] = pd.to_datetime(big["datetime"], utc=True, errors="coerce")
big = big.dropna(subset=["datetime"])

# Make sure numeric
big["cyc"] = pd.to_numeric(big["cyc"], errors="coerce").fillna(0)

# Create hour / weekday / month
big["hour"] = big["datetime"].dt.hour
big["weekday"] = big["datetime"].dt.dayofweek
big["month"] = big["datetime"].dt.month

# Use Dark variable from previous processing
# If Dark not in file, we rebuild it quickly
if "Dark" not in big.columns:
    print("Dark variable not found — rebuild it from solar altitude in earlier notebook.")

Dark variable not found — rebuild it from solar altitude in earlier notebook.


In [3]:
# --- Recompute solar altitude and Dark variable ---

def solar_altitude_deg(dt_utc, lat_deg, lon_deg):
    dts = pd.to_datetime(dt_utc, utc=True)
    n = dts.dt.dayofyear.to_numpy()
    hour = (dts.dt.hour + dts.dt.minute/60.0 + dts.dt.second/3600.0).to_numpy()
    lat = np.deg2rad(lat_deg)
    gamma = 2.0*np.pi/365.0 * (n - 1 + (hour - 12)/24.0)

    decl = (0.006918
            - 0.399912*np.cos(gamma) + 0.070257*np.sin(gamma)
            - 0.006758*np.cos(2*gamma) + 0.000907*np.sin(2*gamma)
            - 0.002697*np.cos(3*gamma) + 0.00148*np.sin(3*gamma))

    eqtime = 229.18*(0.000075
                     + 0.001868*np.cos(gamma) - 0.032077*np.sin(gamma)
                     - 0.014615*np.cos(2*gamma) - 0.040849*np.sin(2*gamma))

    tst = (hour*60.0 + eqtime + 4.0*lon_deg) % 1440.0
    ha = np.deg2rad((tst/4.0) - 180.0)

    cos_zen = np.sin(lat)*np.sin(decl) + np.cos(lat)*np.cos(decl)*np.cos(ha)
    cos_zen = np.clip(cos_zen, -1.0, 1.0)

    zen = np.arccos(cos_zen)
    alt = np.rad2deg(np.pi/2 - zen)
    return alt


# Compute solar altitude per sensor (vectorised by group)
big["solar_altitude_deg"] = np.nan

for sid, g in big.groupby("sensor_id", sort=False):
    lat = float(g["latitude"].iloc[0])
    lon = float(g["longitude"].iloc[0])
    big.loc[g.index, "solar_altitude_deg"] = solar_altitude_deg(g["datetime"], lat, lon)

# Classify
big["light_class"] = np.where(big["solar_altitude_deg"] > 0, "daylight",
                       np.where(big["solar_altitude_deg"] < -6, "darkness", "twilight"))

# Drop twilight
big = big[big["light_class"] != "twilight"].copy()

# Binary variable
big["Dark"] = (big["light_class"] == "darkness").astype(int)

print("Dark variable rebuilt.")
print("Dark=1 proportion:", big["Dark"].mean())

Dark variable rebuilt.
Dark=1 proportion: 0.4665336022488583


In [4]:
SAVE_PATH = "../data/mdm2_data_files/big_table_with_dark.csv"
big.to_csv(SAVE_PATH, index=False)
print("Saved dataset with Dark variable.")

Saved dataset with Dark variable.


In [5]:
# If light_class exists
if "light_class" in big.columns:
    big = big[big["light_class"] != "twilight"].copy()

# Drop missing
big = big.dropna(subset=["cyc"])

print("Rows used:", len(big))
big.head()

Rows used: 478465


,ped,car,cyc,sensor_id,datetime,hour,date_only,dow,longitude,latitude,weekday,month,solar_altitude_deg,light_class,Dark
0,413,381,9,1,2024-01-01 00:00:00+00:00,0,2024-01-01,0,-2.591538,51.453815,0,1,-61.529806,darkness,1
1,402,489,10,1,2024-01-01 01:00:00+00:00,1,2024-01-01,0,-2.591538,51.453815,0,1,-60.225898,darkness,1
2,421,473,10,1,2024-01-01 02:00:00+00:00,2,2024-01-01,0,-2.591538,51.453815,0,1,-54.953852,darkness,1
3,370,419,4,1,2024-01-01 03:00:00+00:00,3,2024-01-01,0,-2.591538,51.453815,0,1,-47.266506,darkness,1
4,132,123,2,1,2024-01-01 04:00:00+00:00,4,2024-01-01,0,-2.591538,51.453815,0,1,-38.404055,darkness,1


In [6]:
mean_cyc = big["cyc"].mean()
var_cyc = big["cyc"].var()

print("\nMean cyclist count:", mean_cyc)
print("Variance cyclist count:", var_cyc)
print("Variance / Mean ratio:", var_cyc / mean_cyc)


Mean cyclist count: 15.929635396528482
Variance cyclist count: 1069.2541926523502
Variance / Mean ratio: 67.12358230655869


In [7]:
poisson_model = smf.glm(
    formula="cyc ~ Dark + C(hour) + C(weekday) + C(month)",
    data=big,
    family=sm.families.Poisson()
).fit()

print(poisson_model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    cyc   No. Observations:               478465
Model:                            GLM   Df Residuals:                   478423
Model Family:                 Poisson   Df Model:                           41
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -6.7250e+06
Date:                Tue, 17 Feb 2026   Deviance:                   1.1995e+07
Time:                        17:32:55   Pearson chi2:                 2.88e+07
No. Iterations:                     6   Pseudo R-squ. (CS):             0.9998
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.0141      0.004    2

In [8]:
pearson_resid = poisson_model.resid_pearson
dispersion = np.sum(pearson_resid**2) / poisson_model.df_resid

print("\nPoisson dispersion statistic:", dispersion)


Poisson dispersion statistic: 60.12878909981651


In [9]:
nb_model = smf.glm(
    formula="cyc ~ Dark + C(hour) + C(weekday) + C(month)",
    data=big,
    family=sm.families.NegativeBinomial()
).fit()

print(nb_model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    cyc   No. Observations:               478465
Model:                            GLM   Df Residuals:                   478423
Model Family:        NegativeBinomial   Df Model:                           41
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.6736e+06
Date:                Tue, 17 Feb 2026   Deviance:                   9.8688e+05
Time:                        17:33:29   Pearson chi2:                 3.47e+06
No. Iterations:                    12   Pseudo R-squ. (CS):             0.4524
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.0313      0.013     

In [10]:
beta_dark = nb_model.params["Dark"]
se_dark = nb_model.bse["Dark"]

percent_change = (np.exp(beta_dark) - 1) * 100

print("\n" + "="*90)
print("RESULTS: DARKNESS EFFECT (CYCLISTS)".center(90))
print("="*90)
print(f"Coefficient (log scale): {beta_dark:.4f}")
print(f"Std. Error: {se_dark:.4f}")
print(f"Estimated % change due to darkness: {percent_change:.2f}%")
print("="*90 + "\n")


                           RESULTS: DARKNESS EFFECT (CYCLISTS)                            
Coefficient (log scale): -0.2585
Std. Error: 0.0071
Estimated % change due to darkness: -22.78%



In [11]:
print("AIC Poisson:", poisson_model.aic)
print("AIC NegBin:", nb_model.aic)

AIC Poisson: 13449989.031330297
AIC NegBin: 3347222.714490086
